# Runtime

## Subject of the exercise

Multi-dimensional hydro simulations can take long!
Days, weeks, month and even years for longterm 3D simulations.

This exercise is supposed to give you an idea of how much longer simulations with higher resolution take.

Also, you will learn why the FARGO algorithm is amazing.

In [1]:
example_name = "runtime"
!mkdir -p $example_name
import os
os.chdir(example_name)

## Setup

We'll take the example setup file from the examples directory and modify it in python.

We remove the planet for now and run only for a very short time. There is an output file `monitor/timestepLogging.dat` 

In [5]:
# make a copy of the default file

src_config = "../../type1.yml"

import ruamel.yaml
yaml = ruamel.yaml.YAML()

config_file = f"setup_ref.yml"

with open(src_config, "r") as infile:
    config = yaml.load(infile)

config["nbody"][1]["mass"] = 2e-5
config["DT"] = 0.314 # monitor scalar files around every half orbit
config["Ninterm"] = 20 # write a snapshot every orbit
config["Ntot"] = 1 # wirte 100 snapshots
# use very low resolution by setting it to 2 cell per scaleheight, cps
config["cps"] = 2
del config["nbody"][1]

config["OutputDir"] = f"output/2cps_fargo"


# write the file to disk
with open(config_file, "w") as outfile:
    yaml.dump(config, outfile)

## Run the simulation

Now we can run all three simulations.

Here is how to start one of them:

In [6]:
cwd = os.getcwd()
cmd = f"cd {cwd} && run_fargo -np 1 -nt 4 auto {config_file}"
print(cmd)

cd /home/thomas/repo/spp1992-fargo-tutorial/notebooks/runtime && run_fargo -np 1 -nt 4 auto setup_ref.yml


## Plotting the results

Investigate the average time one hydro step takes, the number of hydrosteps, and the overall runtime.

You can use the tool to inspect tab separated text files from the repository.

In [7]:
!$(dirname $(which run_fargo))/Tools/inspect_tab_file.py output/2cps_fargo/monitor/timestepLogging.dat

Available variables:
 0   snapshot number
 1   monitor number
 2   hydrostep number
 3   Number of Hydrosteps in last DT
 4   PhysicalTime
 5   walltime
 6   walltime per hydrostep
 7   mean dt
 8   min dt
 9   std dev dt


In [9]:
!$(dirname $(which run_fargo))/Tools/inspect_tab_file.py output/2cps_fargo/monitor/timestepLogging.dat 1 2 5 6

         0	         0	    0.0118	         0
         1	         7	    0.0239	      1.74
         2	        14	    0.0332	      1.31
         3	        21	    0.0429	      1.38
         4	        28	    0.0528	      1.41
         5	        35	    0.0621	      1.33
         6	        42	    0.0714	      1.32
         7	        49	    0.0807	      1.32
         8	        56	      0.09	      1.34
         9	        63	    0.0995	      1.34
        10	        70	     0.109	      1.32
        11	        77	     0.118	      1.32
        12	        84	     0.127	      1.32
        13	        91	     0.136	      1.32
        14	        98	     0.146	      1.39
        15	       105	     0.156	      1.32
        16	       112	     0.167	      1.57
        17	       119	      0.18	       1.9
        18	       126	     0.192	       1.7
        19	       133	     0.201	      1.35
        20	       140	     0.214	      1.85


## Your turn

This is where the exercise starts for you.

Repeat the simulation

- with different resolution (change `cps`) and 
- with and without the FARGO algorithm `Transport: fast` and `Transport: slow`.
- with different numbers of processes and cores `run_fargo -np {number processes} -nt {number threads per process}`
